# Cleaning & Preparing dataset sentiment
Emot is a Emotion Recognition dataset with 5 possible labels: `sadness`, `anger`, `trust`, `fear`, `joy`, 'anticipation', 'surprise', 'disgust'

In [1]:
import os, sys
sys.path.append('../')
os.chdir('../') 

import numpy as np
import pandas as pd

# Prepare Dataset

In [2]:
dataset_path1 = './dataset/dataset_smsa_stok/IDSMSA.csv'
# dataset_path2 = 'dataset/news_bri/train.jsonl'
# dataset_path2 = 'dataset/news_bri/test.jsonl'

In [3]:
import os

print(os.path.exists(dataset_path1))  # harus True


True


In [4]:
dataset1 = pd.read_csv(dataset_path1)
# dataset2 = pd.read_json(dataset_path2, lines=True)
# dataset3 = pd.read_json(dataset_path3, lines=True)

In [5]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288 entries, 0 to 3287
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tweet Date           3288 non-null   object
 1   Sentence             3288 non-null   object
 2   Quote Count          3288 non-null   int64 
 3   Reply Count          3288 non-null   int64 
 4   Retweet Count        3288 non-null   int64 
 5   Favorite Count       3288 non-null   int64 
 6   Sentiment            3288 non-null   object
 7   English Translation  3288 non-null   object
dtypes: int64(4), object(4)
memory usage: 205.6+ KB


In [6]:
dataset1.head(2)

,Tweet Date,Sentence,Quote Count,Reply Count,Retweet Count,Favorite Count,Sentiment,English Translation
0,Thu Feb 29 11:21:27 +0000 2024,"Gk muluk muluk, 100,000 lot saham BBCA aja",0,0,0,0,Positive,"Not too ambitious, just 100,000 lots of BBCA s..."
1,Thu Feb 29 10:11:05 +0000 2024,BCA Expoversary 2024 menawarkan promo suku bun...,0,0,0,0,Neutral,BCA Expoversary 2024 offers special interest r...


In [7]:
dataset1.Sentiment.value_counts()

Sentiment
Positive    1769
Negative     786
Neutral      733
Name: count, dtype: int64

In [8]:
# Ubah ke datetime
dataset1['Tweet Date'] = pd.to_datetime(dataset1['Tweet Date'], format='%a %b %d %H:%M:%S %z %Y')

# Buat kolom baru berisi Tanggal-Bulan-Tahun (bisa dalam string atau datetime)
dataset1['date'] = dataset1['Tweet Date'].dt.strftime('%d %b %Y')

In [9]:
data_smsa =  dataset1[['date','Sentence','Sentiment']]

# Cleaning Data

In [10]:
data_smsa.head(5)

,date,Sentence,Sentiment
0,29 Feb 2024,"Gk muluk muluk, 100,000 lot saham BBCA aja",Positive
1,29 Feb 2024,BCA Expoversary 2024 menawarkan promo suku bun...,Neutral
2,29 Feb 2024,[USERNAME] saham bca nya menyusul ya 🙂,Positive
3,29 Feb 2024,PT Bank BCA Syariah (BCA Syariah) turut memeri...,Neutral
4,29 Feb 2024,[USERNAME] Begitu byk saham kamu memilih saham...,Positive


In [11]:
print(data_smsa['Sentence'].head(10))

0           Gk muluk muluk, 100,000 lot saham BBCA aja
1    BCA Expoversary 2024 menawarkan promo suku bun...
2               [USERNAME] saham bca nya menyusul ya 🙂
3    PT Bank BCA Syariah (BCA Syariah) turut memeri...
4    [USERNAME] Begitu byk saham kamu memilih saham...
5    [USERNAME] Drpd bunga mending hal² yg berguna,...
6    [USERNAME] [USERNAME] Pilihan RDN di Mirae Ass...
7    [USERNAME] [USERNAME] pak buat sekuritas mirae...
8                      [USERNAME] Bbca, soalnya stabil
9    Menggelar pameran masih menjadi salah satu str...
Name: Sentence, dtype: object


In [12]:
# pip install emoji

In [13]:
# Pastikan NLTK sudah mengunduh stopwords dan punktuasi
import re
import nltk
from nltk.corpus import stopwords
import emoji

nltk.download('stopwords')
nltk.download('punkt')

# Inisialisasi stop words untuk bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

def replace_emoji(text):
    return emoji.demojize(text, delimiters=(" ", " "))  # hasil: hello smiling_face

# Fungsi pembersihan utama
def clean_tweet(tweet):
    tweet = tweet.lower()                                 # Lowercase
    tweet = re.sub(r'\[?username\]?', '', tweet, flags=re.IGNORECASE)
    tweet = replace_emoji(tweet)                          # Ganti emoji jadi teks
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet) # Hapus URL
    tweet = re.sub(r'@\w+|#', '', tweet)                  # Hapus mention & hashtag
    tweet = tweet.replace("hashtag", "")                  # Hapus kata "hashtag"
    tweet = re.sub(r'\d+', '', tweet)                     # Hapus angka
    tweet = re.sub(r'[^\w\s]', '', tweet)                 # Hapus tanda baca
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)           # Hapus karakter non-ASCII
    tweet = re.sub(r'\s+', ' ', tweet).strip()            # Hapus spasi berlebih
    return tweet

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter-23524010/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/jupyter-23524010/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
data_smsa['cleaned_Sentence'] = data_smsa['Sentence'].apply(clean_tweet)

/tmp/ipykernel_1947466/1348316642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_smsa['cleaned_Sentence'] = data_smsa['Sentence'].apply(clean_tweet)


In [15]:
# Ubah kolom 'date' ke tipe datetime
data_smsa['date'] = pd.to_datetime(data_smsa['date'], format='%d %b %Y')
# Lakukan pengurutan berdasarkan tanggal (descending / terbaru dulu)
data_smsa = data_smsa.sort_values(by='date', ascending=True).reset_index(drop=True)

/tmp/ipykernel_1947466/1522583709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_smsa['date'] = pd.to_datetime(data_smsa['date'], format='%d %b %Y')


In [16]:
data_smsa.head(5)

,date,Sentence,Sentiment,cleaned_Sentence
0,2020-12-14,[HASHTAG] [HASHTAG] TOP loser by change: DSSA ...,Negative,top loser by change dssa rp poll rp tcpi rp tp...
1,2020-12-17,[HASHTAG] [HASHTAG] TOP loser by change: ITMG ...,Negative,top loser by change itmg rp tcpi rp inkp rp tg...
2,2020-12-18,Rapat Umum Pemegang Saham Luar Biasa (RUPSLB) ...,Positive,rapat umum pemegang saham luar biasa rupslb pt...
3,2021-01-12,"Saham big cap: BBCA, BBNI, BBRI, BMRI, TLKM na...",Positive,saham big cap bbca bbni bbri bmri tlkm naik ha...
4,2021-01-25,"Saham TPIA dan MEDC Masuk Indeks LQ45, Gantika...",Positive,saham tpia dan medc masuk indeks lq gantikan s...


In [17]:
data_smsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288 entries, 0 to 3287
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              3288 non-null   datetime64[ns]
 1   Sentence          3288 non-null   object        
 2   Sentiment         3288 non-null   object        
 3   cleaned_Sentence  3288 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 102.9+ KB


In [18]:
print(data_smsa.isnull().sum())

date                0
Sentence            0
Sentiment           0
cleaned_Sentence    0
dtype: int64


In [19]:
data_smsa.to_csv('dataset/dataset_smsa_stok/data_clean/data_smsa_clean.csv', index=False)

In [20]:
# Menampilkan baris ke-16 dari kolom 'cleaned_tweet'
print(data_smsa.loc[30, 'cleaned_Sentence'])

tpia kembali kantongi restu pemegang saham untuk gelar rights issue url


# Save Data

In [21]:
data_to_save_train_dataset1 = data_smsa[['cleaned_Sentence', 'Sentiment']].copy()

In [22]:
data_to_save_train_dataset1.rename(columns={'cleaned_Sentence': 'text', 'Sentiment': 'sentiment'}, inplace=True)

In [23]:
data_to_save_train_dataset1.to_csv('dataset/dataset_smsa_stok/data_clean/data_train/data_smsa.csv', index=False)

In [24]:
data_to_save_train_dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288 entries, 0 to 3287
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3288 non-null   object
 1   sentiment  3288 non-null   object
dtypes: object(2)
memory usage: 51.5+ KB


## SPLIT DATA

In [25]:
from sklearn.model_selection import train_test_split
train1, valid1 = train_test_split(data_to_save_train_dataset1, test_size=0.2, random_state=42)

In [26]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2630 entries, 2211 to 3174
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       2630 non-null   object
 1   sentiment  2630 non-null   object
dtypes: object(2)
memory usage: 61.6+ KB


In [27]:
train1['sentiment'].value_counts()

sentiment
Positive    1429
Negative     626
Neutral      575
Name: count, dtype: int64

In [28]:
valid1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 658 entries, 2212 to 436
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       658 non-null    object
 1   sentiment  658 non-null    object
dtypes: object(2)
memory usage: 15.4+ KB


In [29]:
valid1['sentiment'].value_counts()

sentiment
Positive    340
Negative    160
Neutral     158
Name: count, dtype: int64

In [30]:
train1.to_csv('dataset/dataset_smsa_stok/data_clean/data_train/data_smsa_train.csv', index=False)
valid1.to_csv('dataset/dataset_smsa_stok/data_clean/data_train/data_smsa_valid.csv', index=False)

In [31]:
# Simpan file dalam format TSV (tab-separated)
train1.to_csv('dataset/dataset_smsa_stok/data_clean/data_train/data_smsa_train.tsv', sep='\t', index=False)
valid1.to_csv('dataset/dataset_smsa_stok/data_clean/data_train/data_smsa_valid.tsv', sep='\t', index=False)

## TRAINING DATA

In [32]:
train_dataset_path = './dataset/dataset_smsa_stok/data_clean/data_train/data_smsa_train.csv'
valid_dataset_path = './dataset/dataset_smsa_stok/data_clean/data_train/data_smsa_valid.csv'

In [33]:
x = pd.read_csv(train_dataset_path)
y = pd.read_csv(valid_dataset_path)
# z = pd.read_csv(test_dataset_path)

In [34]:
print("Train dataset NaN values:")
print(x.isnull().sum())

Train dataset NaN values:
text         0
sentiment    0
dtype: int64


# End Prepare Data